# Filter Change MTPtg/Rotator Integration Test

This notebook performs a filter change scenario integration test between the Maint Telescope pointing component (MTPtg) and the Rotator with the Camera Cable Wrap tracking the Rotator. It includes enough boilerplate to allow the test to run at any time by getting current time information from the pointing and computing appropriate coordinates to slew.

In [1]:
import logging
import yaml

import os
import sqlite3
import pandas as pd

import numpy as np
from matplotlib import pyplot as plt
import astropy.units as u
from astropy.time import Time
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5
import asyncio
from lsst.ts import salobj
from lsst.ts.idl.enums import ATPtg

from astropy.utils import iers
iers.conf.auto_download = False

from datetime import datetime 

In [2]:
test_message = "Filter Change MTPtg_Rotator_CCW Integration Test"
now = datetime.now()
print(test_message, now)

Filter Change MTPtg_Rotator_CCW Integration Test 2019-12-12 17:11:34.622234


In [3]:
con = sqlite3.connect(os.path.expanduser("~/develop/one_filt_v1.4_0yrs_1.db"))

In [4]:
df = pd.read_sql_query("SELECT * from SummaryAllProps", con)

In [5]:
d = salobj.Domain()

In [6]:
script = salobj.Controller("Script", index=1)
rot = salobj.Remote(d, "Rotator")
mtptg = salobj.Remote(d, "MTPtg")
mtm = salobj.Remote(d, "MTMount")

In [7]:
await asyncio.gather(rot.start_task,
                     mtptg.start_task,
                     script.start_task,
                     mtm.start_task)

RemoteTelemetry(MTMount, 0, Camera_Cable_Wrap) falling behind; read 10 messages


[None, None, None, None]

RemoteTelemetry(Rotator, 0, Motors) falling behind; read 13 messages


In [ ]:
await asyncio.sleep(1.)
await salobj.set_summary_state(mtptg, salobj.State.ENABLED)
await salobj.set_summary_state(rot, salobj.State.ENABLED)

In [ ]:
await mtm.cmd_enterControl.start(timeout=10.)
await mtm.cmd_start.start(timeout=10.)
await mtm.cmd_enable.start(timeout=10.)

In [13]:
location = EarthLocation.from_geodetic(lon=-70.747698*u.deg,
                                       lat=-30.244728*u.deg,
                                       height=2663.0*u.m)

In [14]:
def parallactic_angle(location, lst, target):
    """
    Calculate the parallactic angle.
    Parameters
    ----------
    time : `~astropy.time.Time`
        Observation time.
    target : `~astroplan.FixedTarget` or `~astropy.coordinates.SkyCoord` or list
        Target celestial object(s).
    grid_times_targets: bool
        If True, the target object will have extra dimensions packed onto the end,
        so that calculations with M targets and N times will return an (M, N)
        shaped result. Otherwise, we rely on broadcasting the shapes together
        using standard numpy rules.
    Returns
    -------
    `~astropy.coordinates.Angle`
        Parallactic angle.
    Notes
    -----
    The parallactic angle is the angle between the great circle that
    intersects a celestial object and the zenith, and the object's hour
    circle [1]_.
    .. [1] https://en.wikipedia.org/wiki/Parallactic_angle
    """
    # Eqn (14.1) of Meeus' Astronomical Algorithms
    H = (lst - target.ra).radian
    q = np.arctan2(np.sin(H),
                   (np.tan(location.lat.radian) *
                    np.cos(target.dec.radian) -
                    np.sin(target.dec.radian)*np.cos(H)))*u.rad
    return Angle(q)

# Move to 0 deg Starting Position

In [52]:
now = datetime.now()
print("Move to 0.0 deg starting position", now)

alt = 45. * u.deg
az = 0. * u.deg
rot_tel = Angle(0, unit= u.deg) 

target_name="Rotator test"
time_data = await mtptg.tel_timeAndDate.next(flush=True, timeout=2)
curr_time_mtptg = Time(time_data.tai, format="mjd", scale="tai")
time_err = curr_time_mtptg - Time.now()
print(f"Time error={time_err.sec:0.2f} sec")

# Compute RA/Dec for commanded az/el
cmd_elaz = AltAz(alt=alt, az=az, 
                 obstime=curr_time_mtptg.tai, 
                 location=location)
cmd_radec = cmd_elaz.transform_to(ICRS)

# Calculating the other parameters     
rot_pa = rot_tel
print(rot_pa)

await rot.cmd_trackStart.start(timeout=30.)
    
ack = await mtptg.cmd_raDecTarget.set_start(
    targetName=target_name,
    targetInstance=ATPtg.TargetInstances.CURRENT,
    frame=ATPtg.CoordFrame.ICRS,
    epoch=2000,  # should be ignored: no parallax or proper motion
    equinox=2000,  # should be ignored for ICRS
    ra=cmd_radec.ra.hour,
    declination=cmd_radec.dec.deg,
    parallax=0,
    pmRA=0,
    pmDec=0,
    rv=0,
    dRA=0,
    dDec=0,
    rotPA=rot_pa.deg-180,
    rotFrame=ATPtg.RotFrame.FIXED,
    rotMode=ATPtg.RotMode.FIELD,
    timeout=10
)

print("Waiting 30s")
await asyncio.sleep(30.)

RemoteTelemetry(Rotator, 0, Application) falling behind; read 100 messages
RemoteTelemetry(MTMount, 0, Camera_Cable_Wrap) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, timeAndDate) falling behind; read 100 messages


Move to 0.0 deg starting position 2019-12-12 18:04:17.607646


RemoteTelemetry(Rotator, 0, Motors) falling behind; read 100 messages
RemoteTelemetry(Rotator, 0, Electrical) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, mountStatus) falling behind; read 100 messages


Time error=-56.92 sec
0d00m00s


RemoteTelemetry(MTPtg, 0, guidingAndOffsets) falling behind; read 100 messages
RemoteEvent(Rotator, 0, heartbeat) falling behind; read 32 messages
RemoteTelemetry(MTPtg, 0, currentTargetStatus) falling behind; read 100 messages


Waiting 30s


In [53]:
await mtptg.cmd_stopTracking.start(timeout=10.)
await mtm.cmd_stop.start(timeout=10.)
await rot.cmd_stop.start(timeout=10.)

# Start Test

In [54]:
now = datetime.now()
print("Start Test", now)
script.evt_logMessage.set_put(level=logging.INFO+1,
                              message=f"START - {test_message}")

Start Test 2019-12-12 18:05:52.291251


True

# Perform First Track

In [55]:
now = datetime.now()
print("Perform First Track", now)

field = 0
target_name = df["note"][field]
alt = df['altitude'][field] * u.deg
az = df['azimuth'][field] * u.deg
rot_tel = Angle(df["rotTelPos"][field]*u.deg)
para_ang = Angle(df["paraAngle"][field]*u.deg)
rot_pa = para_ang-180*u.deg+rot_tel.wrap_at('180d')
visit_time = df['visitTime'][field]  # how long the visit lasted. Usefull so you can wait between visits
print(f"Alt: {alt}\nAz: {az}\nRotTel: {rot_tel}")

#Here is a small trick to get the ra before transit. Get `timeAndDate` from the pointing component, then use `RA = lst - delta`.
time_data = await mtptg.tel_timeAndDate.next(flush=True, timeout=2)
curr_time_atptg = Time(time_data.tai, format="mjd", scale="tai")

# Compute RA/Dec for commanded az/el
cmd_elaz = AltAz(alt=alt, az=az, obstime=curr_time_atptg.tai, location=location)
cmd_radec = cmd_elaz.transform_to(ICRS) # This is the RA/Dec of the target

print(f"RA: {cmd_radec.ra}\nDec: {cmd_radec.dec}\nRotPA: {rot_pa}")

await rot.cmd_trackStart.start(timeout=30.)

ack = await mtptg.cmd_raDecTarget.set_start(
    targetName=target_name,
    targetInstance=ATPtg.TargetInstances.CURRENT,
    frame=ATPtg.CoordFrame.ICRS,
    epoch=2000,  # should be ignored: no parallax or proper motion
    equinox=2000,  # should be ignored for ICRS
    ra=cmd_radec.ra.hour,
    declination=cmd_radec.dec.deg,
    parallax=0,
    pmRA=0,
    pmDec=0,
    rv=0,
    dRA=0,
    dDec=0,
    rotPA=rot_pa.deg,
    rotFrame=ATPtg.RotFrame.TARGET,
    rotMode=ATPtg.RotMode.FIELD,
    timeout=10.
)

print("Waiting", visit_time, "s")
await asyncio.sleep(visit_time)

RemoteTelemetry(MTMount, 0, Camera_Cable_Wrap) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, currentTargetStatus) falling behind; read 100 messages


Perform First Track 2019-12-12 18:05:52.323841
Alt: 73.57495921073964 deg
Az: 4.273461703999071 deg
RotTel: 25.191637478972904 deg


RemoteTelemetry(Rotator, 0, Motors) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, timeAndDate) falling behind; read 100 messages
RemoteTelemetry(Rotator, 0, Electrical) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, skyEnvironment) falling behind; read 12 messages
RemoteTelemetry(Rotator, 0, Application) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, mountStatus) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, guidingAndOffsets) falling behind; read 100 messages
RemoteEvent(Rotator, 0, heartbeat) falling behind; read 60 messages


RA: 282.6029489328364 deg
Dec: -13.884155902561336 deg
RotPA: 28.993384547298348 deg


RemoteTelemetry(MTPtg, 0, currentTimesToLimits) falling behind; read 12 messages


Waiting 31.0 s


In [56]:
await mtptg.cmd_stopTracking.start(timeout=10.)
await mtm.cmd_stop.start(timeout=10.)
await rot.cmd_stop.start(timeout=10.)

# Perform Filter Change - Move Command

# Perform Filter Change - Track Command

In [57]:
now = datetime.now()
print("Move to 0.0 deg for filter change",now)

field = 1
alt = df['altitude'][field] * u.deg
az = df['azimuth'][field] * u.deg
rot_tel = Angle(0, unit=u.deg)

target_name="filter change"
time_data = await mtptg.tel_timeAndDate.next(flush=True, timeout=2)
curr_time_mtptg = Time(time_data.tai, format="mjd", scale="tai")
time_err = curr_time_mtptg - Time.now()
print(f"Time error={time_err.sec:0.2f} sec")

#Here is a small trick to get the ra before transit. Get `timeAndDate` from the pointing component, then use `RA = lst - delta`.
time_data = await mtptg.tel_timeAndDate.next(flush=True, timeout=2)
curr_time_atptg = Time(time_data.tai, format="mjd", scale="tai")

# Compute RA/Dec for commanded az/el
cmd_elaz = AltAz(alt=alt, az=az, obstime=curr_time_atptg.tai, location=location)
cmd_radec = cmd_elaz.transform_to(ICRS) # This is the RA/Dec of the target

# Calculating the other parameters     
rot_pa = rot_tel
print(rot_pa)

await rot.cmd_trackStart.start(timeout=30.)

ack = await mtptg.cmd_raDecTarget.set_start(
    targetName=target_name,
    targetInstance=ATPtg.TargetInstances.CURRENT,
    frame=ATPtg.CoordFrame.ICRS,
    epoch=2000,  # should be ignored: no parallax or proper motion
    equinox=2000,  # should be ignored for ICRS
    ra=cmd_radec.ra.hour,
    declination=cmd_radec.dec.deg,
    parallax=0,
    pmRA=0,
    pmDec=0,
    rv=0,
    dRA=0,
    dDec=0,
    rotPA=rot_pa.deg-180,
    rotFrame=ATPtg.RotFrame.FIXED,
    rotMode=ATPtg.RotMode.FIELD,
    timeout=10.
)

print("Waiting 30s to move to 0 deg")
await asyncio.sleep(30.)

Move to 0.0 deg for filter change 2019-12-12 18:06:29.539565
Time error=-37.00 sec
0d00m00s
Waiting 30s to move to 0 deg


In [58]:
await mtptg.cmd_stopTracking.start(timeout=10.)
await mtm.cmd_stop.start(timeout=10.)
await rot.cmd_stop.start(timeout=10.)

In [59]:
print("Rotator in position and waiting 90s")
await asyncio.sleep(90.)

Rotator in position and waiting 90s


# Perform Second Track

In [60]:
now = datetime.now()
print("Perform Second Track", now)

target_name = df["note"][field]
#alt = df['altitude'][field] * u.deg
#az = df['azimuth'][field] * u.deg
rot_tel = Angle(df["rotTelPos"][field]*u.deg)
para_ang = Angle(df["paraAngle"][field]*u.deg)
rot_pa = para_ang-180*u.deg+rot_tel.wrap_at('180d')
visit_time = df['visitTime'][field]  # how long the visit lasted. Usefull so you can wait between visits
print(f"Alt: {alt}\nAz: {az}\nRotTel: {rot_tel}")

await rot.cmd_trackStart.start(timeout=30.)

ack = await mtptg.cmd_raDecTarget.set_start(
    targetName=target_name,
    targetInstance=ATPtg.TargetInstances.CURRENT,
    frame=ATPtg.CoordFrame.ICRS,
    epoch=2000,  # should be ignored: no parallax or proper motion
    equinox=2000,  # should be ignored for ICRS
    ra=cmd_radec.ra.hour,
    declination=cmd_radec.dec.deg,
    parallax=0,
    pmRA=0,
    pmDec=0,
    rv=0,
    dRA=0,
    dDec=0,
    rotPA=rot_pa.deg,
    rotFrame=ATPtg.RotFrame.TARGET,
    rotMode=ATPtg.RotMode.FIELD,
    timeout=10.
)

print("Waiting", visit_time, "s")
await asyncio.sleep(visit_time)

Perform Second Track 2019-12-12 18:08:34.452805
Alt: 68.83074605748851 deg
Az: 8.315007012391357 deg
RotTel: 25.832374501812186 deg
Waiting 31.0 s


In [61]:
await mtptg.cmd_stopTracking.start(timeout=10.)
await mtm.cmd_stop.start(timeout=10.)
await rot.cmd_stop.start(timeout=10.)

# Move to Zero - Move Command

# Move to Park Position - Track Command

In [63]:
now = datetime.now()
print("Move to 0.0 deg starting position", now)

alt = 60. * u.deg
az = 0. * u.deg
rot_tel = Angle(0, unit= u.deg) 

target_name="Rotator test"
time_data = await mtptg.tel_timeAndDate.next(flush=True, timeout=2)
curr_time_mtptg = Time(time_data.tai, format="mjd", scale="tai")
time_err = curr_time_mtptg - Time.now()
print(f"Time error={time_err.sec:0.2f} sec")

# Compute RA/Dec for commanded az/el
cmd_elaz = AltAz(alt=alt, az=az, 
                 obstime=curr_time_mtptg.tai, 
                 location=location)
cmd_radec = cmd_elaz.transform_to(ICRS)

# Calculating the other parameters     
rot_pa = rot_tel
print(rot_pa)

await rot.cmd_trackStart.start(timeout=30.)

ack = await mtptg.cmd_raDecTarget.set_start(
    targetName=target_name,
    targetInstance=ATPtg.TargetInstances.CURRENT,
    frame=ATPtg.CoordFrame.ICRS,
    epoch=2000,  # should be ignored: no parallax or proper motion
    equinox=2000,  # should be ignored for ICRS
    ra=cmd_radec.ra.hour,
    declination=cmd_radec.dec.deg,
    parallax=0,
    pmRA=0,
    pmDec=0,
    rv=0,
    dRA=0,
    dDec=0,
    rotPA=rot_pa.deg-180,
    rotFrame=ATPtg.RotFrame.FIXED,
    rotMode=ATPtg.RotMode.FIELD,
    timeout=10
)

print("Waiting 30s")
await asyncio.sleep(30.)

RemoteTelemetry(MTMount, 0, Camera_Cable_Wrap) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, currentTargetStatus) falling behind; read 100 messages
RemoteTelemetry(Rotator, 0, Motors) falling behind; read 100 messages


Move to 0.0 deg starting position 2019-12-12 18:11:03.213633


RemoteTelemetry(Rotator, 0, Electrical) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, timeAndDate) falling behind; read 100 messages
RemoteTelemetry(Rotator, 0, Application) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, skyEnvironment) falling behind; read 23 messages
RemoteEvent(Rotator, 0, heartbeat) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, mountStatus) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, guidingAndOffsets) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, currentTimesToLimits) falling behind; read 23 messages


Time error=-57.14 sec
0d00m00s


RemoteEvent(MTPtg, 0, weatherDataApplied) falling behind; read 11 messages


Waiting 30s


In [64]:
await mtptg.cmd_stopTracking.start(timeout=10.)
await mtm.cmd_stop.start(timeout=10.)
await rot.cmd_stop.start(timeout=10.)

# Test Complete

In [65]:
now = datetime.now()
print("Test Complete",now)

script.evt_logMessage.set_put(level=logging.INFO+1,
                          message=f"END - {test_message}")

Test Complete 2019-12-12 18:11:38.897607


True

# Additional Error Handling Commands

# Bring the Rotator and CCW back to enabled state

In [48]:
await mtm.cmd_clearerror.start(timeout=30)

RemoteTelemetry(MTPtg, 0, currentTargetStatus) falling behind; read 100 messages
RemoteTelemetry(Rotator, 0, Application) falling behind; read 100 messages
RemoteTelemetry(MTMount, 0, Camera_Cable_Wrap) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, timeAndDate) falling behind; read 100 messages
RemoteTelemetry(Rotator, 0, Motors) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, skyEnvironment) falling behind; read 100 messages
RemoteTelemetry(Rotator, 0, Electrical) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, mountStatus) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, guidingAndOffsets) falling behind; read 100 messages
RemoteEvent(Rotator, 0, heartbeat) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, currentTimesToLimits) falling behind; read 100 messages
RemoteEvent(MTPtg, 0, weatherDataApplied) falling behind; read 61 messages


Wait for override off

In [49]:
await rot.cmd_clearError.start(timeout=30)

RemoteTelemetry(MTMount, 0, Camera_Cable_Wrap) falling behind; read 100 messages
RemoteEvent(MTMount, 0, mountState) falling behind; read 38 messages


In [50]:
await salobj.set_summary_state(rot, salobj.State.ENABLED)

[<State.OFFLINE: 4>,
 <State.STANDBY: 5>,
 <State.DISABLED: 1>,
 <State.ENABLED: 2>]

In [51]:
await mtm.cmd_enterControl.start(timeout=30.)
await mtm.cmd_start.start(timeout=30.)
await mtm.cmd_enable.start(timeout=30.)

# Move to sync position with CCW

In [ ]:
print("Move to X.X deg starting position")

await rot.cmd_positionSet.set_start(angle=19.04,timeout=30.)

await rot.cmd_move.start(timeout=30.)

In [ ]:
await rot.cmd_clearError.start(timeout=10)

In [ ]:
await rot.cmd_exitControl.start(timeout=10.)

In [ ]:
await rot.cmd_enterControl.start(timeout=10.)

In [ ]:
await salobj.set_summary_state(rot, salobj.State.ENABLED)

In [24]:
await rot.cmd_stop.start(timeout=10.)

RemoteTelemetry(MTPtg, 0, currentTargetStatus) falling behind; read 100 messages
RemoteTelemetry(MTMount, 0, Camera_Cable_Wrap) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, timeAndDate) falling behind; read 100 messages


In [ ]:
await mtm.cmd_clearerror.start(timeout=10)

In [ ]:
await mtptg.cmd_stopTracking.start(timeout=10)
await mtm.cmd_stop.start(timeout=10)

In [ ]:
await mtm.cmd_stop.start(timeout=10)

Bring CCW Down

In [ ]:
await mtm.cmd_disable.start(timeout=30.)
await mtm.cmd_standby.start(timeout=30.)
await mtm.cmd_exitControl.start(timeout=30.)

Wait for override off

In [ ]:
await mtm.cmd_enterControl.start(timeout=30.)
await mtm.cmd_start.start(timeout=30.)
await mtm.cmd_enable.start(timeout=30.)

In [25]:
await mtptg.cmd_stopTracking.start(timeout=30)

RemoteTelemetry(MTMount, 0, Camera_Cable_Wrap) falling behind; read 100 messages
RemoteTelemetry(Rotator, 0, Application) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, mountStatus) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, guidingAndOffsets) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, currentTargetStatus) falling behind; read 100 messages


In [26]:
await salobj.set_summary_state(mtptg, salobj.State.ENABLED)

[<State.ENABLED: 2>]

RemoteTelemetry(MTMount, 0, Camera_Cable_Wrap) falling behind; read 100 messages
